In [ ]:
%run gravwell-lib.ipynb

# Intro

This notebook demonstrates how you can pull data from Gravwell queries into a Jupyter notebook.

Note the `%run gravwell-lib.ipynb` cell at the top--that imports the necessary code from another notebook residing in the current directory. To use Gravwell in your own notebook, copy that `gravwell-lib.ipynb` into the directory where *your* notebook resides and insert the same code cell at the top.

To pull data from Gravwell, we use the `query` function, which takes two arguments: the Gravwell query to run, and the duration over which to run. The duration is something like "-24h" or "-5m", to indicate "the last 24 hours" or "the last 5 minutes", respectively. You can also specify explicit start and end times, rather than a duration (demonstrated later in this notebook).

The `query` function returns a `pandas.DataFrame` object suitable for plotting and other manipulations.

### About the Demo Server

The queries in this notebook are executed against **demo.gravwell.io** (provided you haven't modified the `GravwellToken` and `GravwellServer` variables in `gravwell-lib.ipynb`).

This server has some basic data in the following tags:

* `pinger`: collects the round-trip time to various hosts on the Internet
* `stocks`: queries the Yahoo stocks API for stock market prices

# Raw Entries
To start with, we might pull back some raw entries from the `stocks` tag, filtering to only show 5 results for the AAPL ticker symbol:

In [ ]:
raw = query("tag=stocks json Ticker==AAPL | limit 5", "-24h")
raw

Note how the DataFrame returned contains a Timestamp, Source, and Tag field in addition to the actual stocks data itself. If we want to see just the data fields, we can iterate over the results and print:

In [ ]:
for x in raw["Data"]:
    print(x)

# Tables
Because the `query` function returns a pandas.DataFrame object, it's already essentially a table and thus we don't really need to do much to display our results. Here is a basic table query run over the last hour:

In [ ]:
table = query("tag=pinger ax | stats mean(ms) as ms stddev(ms) by host ip | table protocol host ip ms stddev", "-1h")
table

# Charts
Now let's run a query using the chart renderer. Note that we call the same function as before: `query`. After running the query, if we just display the `chart` variable, we can see the data points rendered in a table:

In [ ]:
chart = query("tag=pinger ax | stats mean(ms) by host | chart mean by host", "-1h")
chart

To make a chart, we'll use the matplotlib library (imported in `gravwell-lib.ipynb` as `plt`). Here, we indicate that the column named "Timestamp" is the index column, then iterate over all the others to generate plots for each column.

In [ ]:
chart = query("tag=pinger ax | stats mean(ms) by host | chart mean by host", "-24h")
plt.figure(figsize=(12,6))
plt.ylabel("milliseconds")
for column_name in chart.set_index("Timestamp"):
    plt.plot(chart["Timestamp"],chart[column_name],label=column_name)
plt.legend()
plt.show()

If we're not interested in timeseries data, it makes more sense to run our query using the table renderer, then chart the results. Thus we can take the average response times from our earlier query and format it as a bar chart:

In [ ]:
table.set_index("host").plot.bar(y="ms", figsize=(12, 6))

Of course we can also do interesting things like plot error bars using the standard deviation column:

In [ ]:
plt.figure(figsize=(12,6))
plt.ylabel("milliseconds")
plt.errorbar(table["host"],table["ms"],fmt=".",yerr=table["stddev"])
plt.show()

# Specifying Exact Timeframes
Sometimes, "run the query over the last N hours" isn't good enough--you need to run the query over a particular time frame. Maybe you're building a notebook to investigate trends in the stock market between May 8 and May 14, 2022. In this case, you can specify the `start` and `end` parameters to the query function. These should be RFC3339 (ISO 8601) timestamp strings. You can generate those strings by hand, or you can use the datetime library:

In [ ]:
now=datetime.datetime.now().astimezone().isoformat()
now

The `astimezone()` function with no arguments puts the timestamps into the local time zone. If we instead want UTC, we can specify a timezone as an argument:

In [ ]:
start=datetime.datetime.fromisoformat('2022-05-08').astimezone(pytz.utc).isoformat()
end=datetime.datetime.fromisoformat('2022-05-14').astimezone(pytz.utc).isoformat()
start, end

With `start` and `end` variables defined, we can pass them to a query and thus chart the number of `stocks` entries in those dates:

In [ ]:
x = query("tag=stocks json Ticker==AAPL High | stats mean(High) by Ticker | chart mean by Ticker", start=start, end=end)
plt.figure(figsize=(12,6))
plt.ylabel("Price")
plt.plot(x["Timestamp"],x["AAPL"],label="AAPL")
plt.legend()
plt.show()

If we find the "jumps" between trading days to be confusing, we can instead use a scatter plot:

In [ ]:
plt.figure(figsize=(12,6))
plt.ylabel("Price")
plt.scatter(x["Timestamp"],x["AAPL"],label="AAPL")
plt.legend()
plt.show()